1 – Numerical instability: violent development

In [11]:
import sys 
import os   
sys.path.append(os.getcwd() + '/..')
import numpy as np
import math
import matplotlib.pyplot as plt 
import nm_lib as nm
from typing import Type
from matplotlib import animation
import matplotlib
matplotlib.use('TkAgg')

# --- Values reused in other tasks of ex_3 ---
x0 = -2.6 
xf = 2.6
pi = math.pi

CFL = 0.98
a = 1
# ---------------------------

def tfunc(xx):
    return math.cos(6.0 * pi * xx / 5.0) * math.cos(6.0 * pi * xx / 5.0) /  math.cosh(5.0 * xx * xx)

def discretisation(x0, xf, nump, dt_float: Type[float]):
    nint = nump - 1
    dh = dt_float((xf - x0) / nint)
    xx = dt_float(np.arange(nump) * dh + x0) 
    xx_half = dt_float(np.arange(nump) * dh + x0 + dh * 0.5)
    return nint, dh, xx, xx_half

def exactU(xx, hh, nt, a, CFL, bnd_limits, bnd_type: str = 'wrap'):
    if xx.size != hh.size:
        return None
    else:
        t = np.zeros(nt)
        unnt = np.zeros((xx.size, nt))   
        for i in range(0, nt - 1):
            dt = CFL * nm.cfl_adv_burger(a, xx)
            t[i+1] = t[i] + dt
            r = a * t[i+1]
            for j in range(0, xx.size):
                unnt[j, i] = tfunc((xx[j] - x0 - r) % (xf - x0) + x0) 
            tmp = unnt[bnd_limits[0]:hh.size-bnd_limits[1], i]
            unnt[:, i] = np.pad(tmp, bnd_limits, bnd_type) 
        return t, unnt

def compare(nump, nt, a, CFL, ddx = lambda x,y: nm.deriv_fwd(x, y), bnd_limits: list=[0,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc(xx[i])

    t, unnt = nm.evolv_adv_burgers(xx, hh, nt, a, CFL, ddx, 'wrap', bnd_limits)
    t_exact, unnt_exact = exactU(xx, hh, nt, a, CFL, bnd_limits, 'wrap')

    # Using pyplot for animation
    plt.figure()
    plt.xlim(xx[0], xx[nump-1])
    plt.ylim(unnt_exact.min() - 0.1 * (unnt_exact.max() - unnt_exact.min()), unnt_exact.max() + 0.1 * (unnt_exact.max() - unnt_exact.min()))
    plt.xlabel("x")
    plt.ylabel("u") 
    line1, = plt.plot([], [], label='Exact solution')
    line2, = plt.plot([], [], '--', label='Numerical solution')

    plt.legend(loc='upper right')
    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
 
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        time_text.set_text('')
        return line1, line2
 
    def update(frame):
        line1.set_data(xx, unnt_exact[:,frame])  
        line2.set_data(xx, unnt[:,frame])   
        time_text.set_text("Time = " + str(t[frame]))
        return line1, line2
 
    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size-1, blit=True)
    plt.show(block=True)

limits = [1, 0] # becomes [0, 1] when using deriv_fwd
nt = 30
#for nump in (65, 129, 1025):
#    compare(nump, int(nt*(nump-1)/64), a, CFL, ddx=nm.deriv_bck, bnd_limits=limits) # Using deriv_fwd or deriv_bck to see different behaviours

2 – Centered differences

In [12]:
limits = [1, 1]
nt = 10

#for nump in (65, 129, 257):
#    compare(nump, int(nt*(nump-1)/64), a, CFL, ddx=nm.deriv_cent, bnd_limits=limits) # previous gifs overwritten

CFL = 0.3
#for nump in (65, 129, 257):
#    compare(nump, int(nt*(nump-1)/64)*3, a, CFL, ddx=nm.deriv_cent, bnd_limits=limits) # previous gifs overwritten

3 – The stability of the non-centered finite-differences schemes


_For simplicity, below the same number of time steps are used for different CFL numbers, but the numerical behaviours are rather clear. When CFL=0.5, there is no instability but strong diffusion. When CFL=0.99 or 1.01, the resutls are similar but the former one leads to minor diffusion and the later on leads to minor instability, indicated by the increasing amplitude. When CFL=2, the instability completely spoils the result._

In [13]:
nump = 129
limits = [1, 0]
nt = 100
for CFL in (0.5, 0.99, 1.01, 2.0):
    compare(nump, nt, a, CFL, ddx=nm.deriv_bck, bnd_limits=limits) 

can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


4- Optional (A)

In [14]:
x0 = -1.4 
xf = 2.0
xc = 0.7
W = 0.1
B = 0.3

def tfunc_Burgers(xx, A):
    return A * (math.tanh((xx + xc) / W) - math.tanh((xx - xc) / W)) + B

def run_Burgers(nump, nt, CFL, A, ddx = lambda x,y: nm.deriv_fwd(x, y), bnd_limits: list=[0,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc_Burgers(xx[i], A)
 
    t, unnt = nm.evolv_uadv_burgers(xx, hh, nt, CFL, ddx, 'wrap', bnd_limits)
    
    plt.figure()
    plt.xlim(xx[0], xx[nump-1])
    plt.ylim(unnt[:,0].min() - 0.1 * (unnt[:,0].max() - unnt[:,0].min()), unnt[:,0].max() + 0.1 * (unnt[:,0].max() - unnt[:,0].min()))
    plt.xlabel("x")
    plt.ylabel("u") 
    line, = plt.plot([], [], '--', label='Numerical solution of the nonlinear Burgers equation: A='+str(A))

    plt.legend(loc='lower right')
    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
 
    def init():
        line.set_data([], [])
        time_text.set_text('')
        return line,
 
    def update(frame):
        line.set_data(xx, unnt[:,frame]) 
        time_text.set_text("Time = " + str(t[frame]))
        return line, 
 
    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size-1, blit=True)
    plt.show(block=True)

nump = 129
limits = [1, 0] 
CFL = 0.98
A = 1.0
nt = 20
#run_Burgers(nump, nt, CFL, A, ddx=nm.deriv_bck, bnd_limits=limits)  
A = -0.02
nt = 1000
#run_Burgers(nump, nt, CFL, A, ddx=nm.deriv_bck, bnd_limits=limits)  

4- Optional (B) 

_When using forward differencing, numerical instability appears soon. Moreover, because the time step depends on the maximum local speed, time steps drastically decrease when the maximum speed increases, and thus the wave propagation becomes invisible._

In [15]:
nump = 257
limits = [0, 1] 
CFL = 0.98
A = 1.0
nt = 20
run_Burgers(nump, nt, CFL, A, ddx=nm.deriv_fwd, bnd_limits=limits)  

def run_2Burgers(nump, nt, CFL, A, ddx1 = lambda x,y: nm.deriv_fwd(x, y), ddx2 = lambda x,y: nm.deriv_bck(x, y), bnd_limits: list=[0,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc_Burgers(xx[i], A)
 
    t1, unnt1 = nm.evolv_uadv_burgers(xx, hh, nt, CFL, ddx1, 'wrap', bnd_limits)
    t2, unnt2 = nm.evolv_uadv_burgers(xx, hh, nt, CFL, ddx2, 'wrap', bnd_limits)

    plt.figure()
    plt.xlim(xx[0], xx[nump-1])
    plt.ylim(unnt1.min() - 0.1 * (unnt1.max() - unnt1.min()), unnt1.max() + 0.1 * (unnt1.max() - unnt1.min()))
    plt.xlabel("x")
    plt.ylabel("u") 
    line1, = plt.plot([], [], '--', label='Numerical solution 1')
    line2, = plt.plot([], [], '--', label='Numerical solution 2')
    line3, = plt.plot([], [], '--', label='Initial solution')

    plt.legend(loc='lower right')
    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
 
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        line3.set_data([], [])
        time_text.set_text('')
        return line1, line2
 
    def update(frame):
        line1.set_data(xx, unnt1[:,frame])  
        line2.set_data(xx, unnt2[:,frame]) 
        line3.set_data(xx, unnt1[:,0]) 
        time_text.set_text("Time 1 = "+str(t1[frame]))
        return line1, line2
 
    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t1.size-1, blit=True)
    plt.show(block=True)

nt = 3
nump = 65
run_2Burgers(nump, nt, CFL, A, ddx1=nm.deriv_fwd, ddx2=nm.deriv_bck, bnd_limits=limits)   

can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


: 